In [1]:
import tensorflow as tf

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [2]:
tf.__version__

'1.14.0'

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10807713568318899882
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 2622182079110820315
physical_device_desc: "device: XLA_GPU device"
, name: "/device:XLA_GPU:1"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 4364576654202505950
physical_device_desc: "device: XLA_GPU device"
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 9336564850093370598
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 7966438196
locality {
  bus_id: 1
  links {
    link {
      device_id: 1
      type: "StreamExecutor"
      strength: 1
    }
  }
}
incarnation: 592305377500309145
physical_device_desc: "device: 0, name: GeForce GTX 1080, pci bus id: 0000:03:00.0, compute capability: 6.1"
, name: "/device:GPU:1"
device_type: "GPU"
mem

In [4]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

Using TensorFlow backend.


['/job:localhost/replica:0/task:0/device:GPU:0',
 '/job:localhost/replica:0/task:0/device:GPU:1']

In [5]:
import math
import numpy as np
import tensorflow as tf

from keras.layers import Input, Bidirectional, LSTM, Embedding, Dense, Dropout, Lambda, Concatenate, CuDNNLSTM
from keras.models import Model
from keras.layers.wrappers import TimeDistributed
from keras.utils import to_categorical
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from model.callbacks import F1score
from keras.backend.tensorflow_backend import set_session
from model.data_utils import minibatches, pad_sequences
from model.crf_layer import ChainCRF
from keras import optimizers
from keras.callbacks import LearningRateScheduler
import matplotlib.pyplot as plt
from model.callbacks import LossHistory

In [6]:
class BaseKerasModel(object):
    """Generic class for general methods that are not specific to NER"""

    def __init__(self, config):
        """Defines self.config and self.logger
        Args:
            config: (Config instance) class with hyper parameters,
                vocab and embeddings
        """
        self.config = config
        self.logger = config.logger
        self.model = None
        self.conf = tf.ConfigProto()
        self.conf.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
        self.conf.log_device_placement = True  # to log device placement (on which device the operation ran)
        self.sess = tf.Session(config=self.conf)
        set_session(self.sess)
        self.saver = None

    def batch_iter(self, train, batch_size, return_lengths=False):
        """
        Creates a batch generator for the dataset
        :param train: Dataset
        :param batch_size: Batch Size
        :param return_lengths: If True, generator returns sequence lengths. Used masking data during the evaluation step
        :return: (number of batches in dataset, data generator)
        """
        nbatches = (len(train) + batch_size - 1) // batch_size

        def data_generator():
            while True:
                for i, (words, labels) in enumerate(minibatches(train, batch_size)):

                    # word_ids = zip(*words)
                    word_ids, sequence_lengths = pad_sequences(words, 0, max_length=self.config.max_length)

                    if labels:
                        labels, _ = pad_sequences(labels, 0, max_length=self.config.max_length)
                        labels = [to_categorical(label, num_classes=self.config.ntags) for label in labels] # Change labels to one-hot

                    # build dictionary
                    inputs = {
                        "word_ids": np.asarray(word_ids),
                    }

                    if return_lengths:
                        yield(inputs, np.asarray(labels), sequence_lengths)

                    else:
                        yield (inputs, np.asarray(labels))

        return (nbatches, data_generator())

    def train(self, train, dev, show_history=False):
        batch_size = self.config.batch_size

        nbatches_train, train_generator = self.batch_iter(train, batch_size)
        nbatches_dev, dev_generator = self.batch_iter(dev, batch_size)


        _, f1_generator = self.batch_iter(dev, batch_size, return_lengths=True)
        f1 = F1score(f1_generator, nbatches_dev, self.run_evaluate)

        callbacks = self.gen_callbacks([f1])

        history = self.model.fit_generator(generator=train_generator,
                                           steps_per_epoch=nbatches_train,
                                           validation_data=dev_generator,
                                           validation_steps=nbatches_dev,
                                           epochs=self.config.nepochs,
                                           callbacks=callbacks) #, nbatches_train

        if show_history:
            print(history.history['f1'])
            pass


    def predict_words(self, words_raw):
        words = [self.config.processing_word(w) for w in words_raw]
        # if type(words[0]) == tuple:
        #     words = zip(*words)
        # char_ids, word_ids = words

        word_ids = np.asarray(words)
        s = word_ids.shape
        word_ids = word_ids.reshape(-1, s[0])
        inputs = [word_ids]

        one_hot_preds = self.model.predict_on_batch(inputs)
        #print("One hot preds: ", one_hot_preds)
        one_hot_preds = [a.flatten() for a in one_hot_preds.squeeze()] #Squeeze to remove unnecessary 1st dimension for batch size
        #print("One hot preds: ", one_hot_preds)

        pred_ids = np.argmax(one_hot_preds, axis=1)
        #print("Pred ids: ", pred_ids)

        preds = [self.idx_to_tag[idx] for idx in pred_ids]

        return preds

    def run_evaluate(self, data_generator, steps_per_epoch):
        accs = []
        label_true = []
        label_pred = []
        for i in range(steps_per_epoch):
            #try:
            x_true, y_true, sequence_lengths = next(data_generator)
            y_pred = self.model.predict_on_batch(x_true)

            for lab, lab_pred, length in zip(y_true, y_pred,
                                             sequence_lengths):
                lab = lab[:length]
                lab_pred = lab_pred[:length]

                lab = np.argmax(lab, axis=1)
                lab_pred = np.argmax(lab_pred, axis=1)
                accs += [a==b for (a, b) in zip(lab, lab_pred)]


                label_true.extend(lab)
                label_pred.extend(lab_pred)


        label_true = np.asarray(label_true)
        #print("Truths: ", label_true)
        label_pred = np.asarray(label_pred)
        #print("Preds: ", label_pred)

        acc = np.mean(accs)
        print("acc: ", 100*acc)

        micro_score = f1_score(label_true, label_pred, average='micro')
        print(' - micro f1: {:04.2f}'.format(micro_score * 100))

        macro_score = f1_score(label_true, label_pred, average='macro')
        print(' - macro f1: {:04.2f}'.format(macro_score * 100))

        weighted_score = f1_score(label_true, label_pred, average='weighted')
        print(' - weighted f1: {:04.2f}'.format(weighted_score * 100))

        print(classification_report(label_true, label_pred))
        return (micro_score, macro_score, weighted_score)

    def get_loss(self):
        return self._loss

    def __getattr__(self, name):
        return getattr(self.model, name)

    def get_optimizer(self):
        return self._optimizer

In [7]:
class BLSTMCRF(BaseKerasModel):

    def __init__(self, config):
        super(BLSTMCRF, self).__init__(config)
        self._loss = None #losses.sparse_categorical_crossentropy
        self._optimizer = optimizers.Nadam(lr=self.config.lr)
        self.idx_to_tag = {idx: tag for tag, idx in
                           self.config.vocab_tags.items()}

    def build(self):
        inputs = [] #Create input for Model

        # build word embeddings
        input_words = Input(shape=(None,), dtype='int32', name='word_ids')
        inputs.append(input_words)
        if self.config.embeddings is None:
            word_embeddings = Embedding(input_dim=self.config.nwords,
                                        output_dim=self.config.dim_word,
                                        mask_zero=True,
                                        name="word_embeddings")(input_words)
        else:
            word_embeddings = Embedding(input_dim=self.config.nwords,
                                        output_dim=self.config.dim_word,
                                        mask_zero=False,
                                        weights=[self.config.embeddings],
                                        trainable=self.config.train_embeddings,
                                        name="word_embeddings")(input_words)

        word_embeddings = Dropout(self.config.dropout)(word_embeddings)
        encoded_text = Bidirectional(CuDNNLSTM(units=self.config.hidden_size_lstm, return_sequences=True),
                                     name="bidirectional_1")(word_embeddings)
        encoded_text = Dropout(self.config.dropout)(encoded_text)
        encoded_text = Bidirectional(CuDNNLSTM(units=self.config.hidden_size_lstm, return_sequences=True),
                                     name="bidirectional_2")(encoded_text)
        encoded_text = Dropout(self.config.dropout)(encoded_text)
        encoded_text = TimeDistributed(Dense(self.config.ntags))(encoded_text)
        encoded_text = Dropout(self.config.dropout)(encoded_text)
        #encoded_text = Dense(100, activation='tanh')(encoded_text)

        if self.config.use_crf:
#             crf = CRF(self.config.ntags, sparse_target=False)
            crf = ChainCRF()
            self._loss = crf.loss
#             self._loss = crf.loss_function
            pred = crf(encoded_text)

        else:
            self._loss = 'categorical_crossentropy'
            pred = Dense(self.config.ntags, activation='softmax')(encoded_text)

        self.model = Model(inputs, pred)


    def gen_callbacks(self, callbacks_list):
        lrate = LearningRateScheduler(self.step_decay)
        callbacks_list.append(lrate)

        #loss_history = LossHistory(self.step_decay)
        #callbacks_list.append(loss_history)
        return callbacks_list


    def step_decay(self, epoch):
        initial_lrate = self.config.lr
        decay = self.config.lr_decay
        epochs_drop = self.config.epoch_drop
        lrate = initial_lrate * math.pow(decay,
                                         math.floor((1+epoch)/epochs_drop))
        return lrate

    def plot_history(self, history):
        plt.plot(history.losses)
        plt.title('model losses')
        plt.ylabel('Loss')
        plt.xlabel('epoch')
        plt.show()

In [8]:
from model.data_utils import Dataset
from model.config import Config
from keras.models import load_model

config = Config()

In [9]:
# build model
model = BLSTMCRF(config)
model.build()
model.compile(optimizer=model.get_optimizer(), loss=model.get_loss())  # , metrics=['categorical_accuracy']
print(model.summary())

From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_ops.py:2403: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
word_ids (InputLayer)        (None, None)              0         
_________________________________________________________________
word_embeddings (Embedding)  (None, None, 50)          5512550   
_________________________________________________________________
dropout_1 (Dropout)          (None, None, 50)          0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, None, 300)         242400    
_________________________________________________________________
dropout_2 (Dropout)          (None, None, 300)         0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, None, 300)         542400    
_________________________________________________________________
dropout_3 (Dropout)          (None, None, 300)         0         
__________

In [10]:
# create datasets
dev = Dataset(config.filename_dev, config.processing_word,
              config.processing_tag, config.max_iter)
train = Dataset(config.filename_train, config.processing_word,
                config.processing_tag, config.max_iter)

In [ ]:
# train model
model.train(train, dev)

Epoch 1/8
23170/23170 [==============================] - 2793s 121ms/step - loss: 3.6950 - val_loss: 4.2269
acc:  93.60303105531511
 - micro f1: 93.60
 - macro f1: 75.31
 - weighted f1: 92.85
              precision    recall  f1-score   support

           0       0.77      0.35      0.49   1161349
           1       0.94      0.99      0.96  21617464
           2       0.93      0.72      0.81   2171441

    accuracy                           0.94  24950254
   macro avg       0.88      0.69      0.75  24950254
weighted avg       0.93      0.94      0.93  24950254

Epoch 2/8
23170/23170 [==============================] - 2792s 120ms/step - loss: 3.1016 - val_loss: 4.1099
acc:  94.35621777638015
 - micro f1: 94.36
 - macro f1: 77.68
 - weighted f1: 93.70
              precision    recall  f1-score   support

           0       0.80      0.37      0.51   1161349
           1       0.95      0.99      0.97  21617464
           2       0.94      0.78      0.86   2171441

    accuracy     

In [ ]:
import os
from ast import literal_eval
import pandas as pd
import re
from seqeval.metrics import classification_report

In [ ]:
# Create directory for storing model object
if not os.path.exists('./results/saves'):
    os.makedirs('./results/saves')

# Save model
model.save_weights('./results/saves/model.h5')

In [ ]:
print("Model Trained and saved")

In [16]:
model